In [9]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.append(str(Path("../../src").resolve()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from utils.openai_lm import OpenAIModel
from utils.parser import load_prompt, load_jsonl, save_jsonl

In [11]:
from utils.openai_lm import OpenAIModel
from utils.together_lm import TogetherModel

# model_name = "gpt-4-0314"
# model_name = "gpt-35-turbo-0301"
# model_name = "togethercomputer/llama-2-7b-chat"
model_name = "togethercomputer/llama-2-70b-chat"

if "gpt" in model_name:
    model = OpenAIModel(model_name=model_name, cache_file=None, model_mode="chat_gpt", api_type="azure")
else:
    model = TogetherModel(model_name=model_name, cache_file=None)

# output, response = model.generate("Hello, how are you?")
output, response = model.generate([{"role": "user", "content": "Hello, how are you?"}])
print(output)



It's a pleasure to meet you.

What brings you here today?

I'm happy to help you with any questions you have.

Is there something specific you'd like to know or discuss?

I'm all ears!

Feel free to ask me anything.

I'm here to assist you in any way I can.

What's on your mind?

I'm looking forward to our


In [12]:
prompt_path = "prompt.txt"
# inst_path = f"../../scripts/step1_generate_tasks/outputs/instructions.{model_name}.1000.txt"
inst_path = f"../../scripts/step1_generate_instructions/outputs/instructions.gpt-4-0314.1000.txt"

prompt_template = load_prompt(prompt_path)
inst_list = load_jsonl(inst_path)
print(f"Number of instructions: {len(inst_list)}")

Number of instructions: 1000


In [13]:
import random
num_inst = len(inst_list)
num_inst = 100
random.seed(42)
inst_list = random.sample(inst_list, num_inst)
print(f"Number of instructions: {len(inst_list)}")

Number of instructions: 100


In [14]:

for inst in inst_list:
    # print(para)
    instruction = inst["instruction"]
    prompt = prompt_template.substitute(instruction=instruction)
    inst["input_messages"] = [{"role": "user", "content": prompt}]
    # input_messages_list.append([{"role": "user", "content": prompt}])
    # task_list.append(task)
    # print(prompt)

In [15]:
from tqdm import tqdm
import re

# output_path = f"outputs/instructions_score.{model_name}.{num_inst}.txt"
output_path = f'outputs/instructions.gpt-4-0314.1000.score.{model_name.split("/")[-1]}.txt'
output_list = []
for i, inst in enumerate(tqdm(inst_list)):
    try: 
        input_messages = inst["input_messages"]
        output, response = model.generate(input_messages, max_tokens=2048, temperature=0.0)
        # match Analysis: (xxx)\nScore: (xxx)
        match = re.search(r"Analysis: (.*)\nScore: ([0-9]+)", output)
        if not match:
            score = 0
        else:
            score = int(match.group(2))
        output_list.append({"score": score, "instruction": inst["instruction"], "context": inst["context"]})
    except Exception as e:
        print(e)
    if i % 100 == 0:
        save_jsonl(output_list, output_path)
        print(f"Number of outputs: {len(output_list)}")
save_jsonl(output_list, output_path)
print(f"Number of outputs: {len(output_list)}")

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:09<15:54,  9.65s/it]

Number of outputs: 1


 58%|█████▊    | 58/100 [12:22<11:09, 15.94s/it]ERROR:root:API error: 504 Server Error: Gateway Time-out for url: https://api.together.xyz/api/inference (0). Waiting 1 sec
ERROR:root:API error: 504 Server Error: Gateway Time-out for url: https://api.together.xyz/api/inference (1). Waiting 2 sec
 67%|██████▋   | 67/100 [18:20<25:04, 45.58s/it]ERROR:root:API error: 504 Server Error: Gateway Time-out for url: https://api.together.xyz/api/inference (0). Waiting 1 sec
ERROR:root:API error: 504 Server Error: Gateway Time-out for url: https://api.together.xyz/api/inference (1). Waiting 2 sec
 88%|████████▊ | 88/100 [28:59<05:22, 26.85s/it]ERROR:root:API error: 504 Server Error: Gateway Time-out for url: https://api.together.xyz/api/inference (0). Waiting 1 sec
ERROR:root:API error: 504 Server Error: Gateway Time-out for url: https://api.together.xyz/api/inference (1). Waiting 2 sec
100%|██████████| 100/100 [36:33<00:00, 21.93s/it]

Number of outputs: 100
